## Model Interpreter Example Notebook

### Setup

In [1]:
import xgboost as xgb

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

import sys

sys.path.append("../../")

import model_interpreter
from model_interpreter.interpreter import ModelInterpreter

In [2]:
model_interpreter.__version__

'1.0.0'

### Prepare data

In [3]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [5]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)

In [6]:
feature_names = list(X.columns)

In [7]:
print(feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


### Train model

In [8]:
xgb_model = xgb.train(
    params={"seed": 1, "max_depth": 6, "min_child_weight": 20}, dtrain=dtrain
)

### Build single model contribution object

A model interpreter object is created by specifiying the feature names in the same order as used for building the model.

In [9]:
single_model_contribution = ModelInterpreter(feature_names)

### Build explainer

The fit method needs to be called on the model to build the explainer.

In [10]:
single_model_contribution.fit(xgb_model)

### Define single row of data to get contributions for

In [11]:
single_row = X_test[feature_names].head(1)

### Use the transform method to get feature contributions

The transform method is called on the single row to create the class output, which by default returns a sorted list of dictionaries of features with descending absolute contribution to the prediction in the format:

`[{feature_name: feature_contribution}, ... ]`

return_feature_values can also be set to True so the return is:

`[{feature_name: (feature_value, feature_contribution)}, ... ]`

This only works if return_type is set to any value except the default of 'name_value_dicts'

In [12]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=False
)
print(contribution_list)

[{'Name': 'MedInc', 'Value': -0.7173248529}, {'Name': 'Latitude', 'Value': -0.2954589427}, {'Name': 'AveOccup', 'Value': -0.2584415674}, {'Name': 'Longitude', 'Value': -0.194666937}, {'Name': 'AveRooms', 'Value': 0.0455645546}, {'Name': 'HouseAge', 'Value': -0.0052499785}, {'Name': 'AveBedrms', 'Value': -0.0039967196}, {'Name': 'Population', 'Value': -0.0015882736}]


In [13]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=True, return_type="dicts"
)
print(contribution_list)

[{'MedInc': (1.6812, -0.71732485)}, {'Latitude': (36.06, -0.29545894)}, {'AveOccup': (3.8774373259052926, -0.25844157)}, {'Longitude': (-119.01, -0.19466694)}, {'AveRooms': (4.192200557103064, 0.045564555)}, {'HouseAge': (25.0, -0.0052499785)}, {'AveBedrms': (1.0222841225626742, -0.0039967196)}, {'Population': (1392.0, -0.0015882736)}]


You can provide a `feature_mapping` dictionary which can either map feature names to more interpretable names, or group features together

In [14]:
mapping_dict = {
    "MedInc": "MedInc feature",
    "HouseAge": "Age of house",
    "AveRooms": "Average number of rooms",
    "AveBedrms": "Average number of bedrooms",
    "Population": "Population feature",
    "AveOccup": "Average occupation",
    "Latitude": "Lattitude of location",
    "Longitude": "Longitude of location",
}

contribution_list_mapped = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict
)
print(contribution_list_mapped)

[{'Name': 'MedInc feature', 'Value': -0.7173248529}, {'Name': 'Lattitude of location', 'Value': -0.2954589427}, {'Name': 'Average occupation', 'Value': -0.2584415674}, {'Name': 'Longitude of location', 'Value': -0.194666937}, {'Name': 'Average number of rooms', 'Value': 0.0455645546}, {'Name': 'Age of house', 'Value': -0.0052499785}, {'Name': 'Average number of bedrooms', 'Value': -0.0039967196}, {'Name': 'Population feature', 'Value': -0.0015882736}]


In the below example we create groups for the number of rooms and location. The resulting grouped contributions equal the sum of the individual feature contributions.

In [15]:
grouping_dict = {
    "MedInc": "MedInc feature",
    "HouseAge": "Age of house",
    "AveRooms": "Number of rooms",
    "AveBedrms": "Number of bedrooms",
    "Population": "Population feature",
    "AveOccup": "Average occupation",
    "Latitude": "Location",
    "Longitude": "Location",
}

contribution_list_grouped = single_model_contribution.transform(
    single_row, feature_mappings=grouping_dict
)
print(contribution_list_grouped)

[{'Name': 'MedInc feature', 'Value': -0.7173248529}, {'Name': 'Location', 'Value': -0.4901258796}, {'Name': 'Average occupation', 'Value': -0.2584415674}, {'Name': 'Number of rooms', 'Value': 0.0455645546}, {'Name': 'Age of house', 'Value': -0.0052499785}, {'Name': 'Number of bedrooms', 'Value': -0.0039967196}, {'Name': 'Population feature', 'Value': -0.0015882736}]


There are also three `sorting` options avaliable:
- `'abs'`,  which is the default used in the above examples sorts by the absolute value of the feature contribution
- `'positive'`, which sorts the contributions in descending order
- `'label'`, which sorts in a descending order if `pred_label > 0`, and ascending if `pred_label = 0`

 `n_return` can also be specified to return only the top n features according to the sorting option applied. 
 
 Some examples of how these variables are used are provided below.

In [16]:
contribution_list_abs = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict, sorting="abs", n_return=5
)
print(contribution_list_abs)

[{'Name': 'MedInc feature', 'Value': -0.7173248529}, {'Name': 'Lattitude of location', 'Value': -0.2954589427}, {'Name': 'Average occupation', 'Value': -0.2584415674}, {'Name': 'Longitude of location', 'Value': -0.194666937}, {'Name': 'Average number of rooms', 'Value': 0.0455645546}]


In [17]:
contribution_list_label_pos = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict, sorting="label", pred_label=1
)
print(contribution_list_label_pos)

[{'Name': 'Average number of rooms', 'Value': 0.0455645546}, {'Name': 'Population feature', 'Value': -0.0015882736}, {'Name': 'Average number of bedrooms', 'Value': -0.0039967196}, {'Name': 'Age of house', 'Value': -0.0052499785}, {'Name': 'Longitude of location', 'Value': -0.194666937}, {'Name': 'Average occupation', 'Value': -0.2584415674}, {'Name': 'Lattitude of location', 'Value': -0.2954589427}, {'Name': 'MedInc feature', 'Value': -0.7173248529}]


In [18]:
contribution_list_label_0 = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict, sorting="label", pred_label=0
)
print(contribution_list_label_0)

[{'Name': 'MedInc feature', 'Value': -0.7173248529}, {'Name': 'Lattitude of location', 'Value': -0.2954589427}, {'Name': 'Average occupation', 'Value': -0.2584415674}, {'Name': 'Longitude of location', 'Value': -0.194666937}, {'Name': 'Age of house', 'Value': -0.0052499785}, {'Name': 'Average number of bedrooms', 'Value': -0.0039967196}, {'Name': 'Population feature', 'Value': -0.0015882736}, {'Name': 'Average number of rooms', 'Value': 0.0455645546}]


We can also chose to return a single dictionary with format `{feature_name: feature_contribution, ... }` or list of tuples with format `[(feature_name, feature_contribution),  ... ]` using the `return_type` variable 

In [19]:
contribution_single_dict = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict, n_return=5, return_type="single_dict"
)
print(contribution_single_dict)

{'MedInc feature': -0.71732485, 'Lattitude of location': -0.29545894, 'Average occupation': -0.25844157, 'Longitude of location': -0.19466694, 'Average number of rooms': 0.045564555}


In [20]:
contribution_list_tups = single_model_contribution.transform(
    single_row, feature_mappings=mapping_dict, n_return=5, return_type="tuples"
)
print(contribution_list_tups)

[('MedInc feature', -0.71732485), ('Lattitude of location', -0.29545894), ('Average occupation', -0.25844157), ('Longitude of location', -0.19466694), ('Average number of rooms', 0.045564555)]
